# pandas cheat sheet

*No BS, just essential pandas!*

**By Ville Voutilainen/NoobQuant**

Cheat sheet to data wrangling with Python pandas. The aim is to cover essential tips and tricks for data workflow. Each topic is featured with a minimal working example.

Tested on NoobQuant conda environments *dev2021* (pandas version 1.1.3). Best viewed on [nbviewer](https://nbviewer.jupyter.org/).

Proper Table of Contents coming soon.

In [1]:
############################################################
# Run this cell!
# Dependencies, auxiliary functions etc.
############################################################
import os
import pandas as pd
import numpy as np
from IPython.display import display_html
from IPython.display import display
import datetime
import random
print("pandas vesion {}".format(pd.__version__))

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    html_str = html_str.replace("table","table style='display:inline; border:0px'")          
    display_html(html_str,raw=True)

pandas vesion 1.1.3


# Input/output data

## Excel files

### Import Excel file

In [2]:
dtype = {
    "col1":np.float64,
    "col3":object,
    "col5":int,
}
df = pd.read_excel(
    io="example_data/excel_import_test.xlsx",
    sheet_name="Sheet1",
    skiprows=4,
    usecols=("A,B,D:F"),
    header=0,
    index_col=0,
    dtype=dtype,
)
print(df.dtypes)
df.head()

col1    float64
col3     object
col4    float64
col5      int32
dtype: object


,col1,col3,col4,col5
2020M1,0.102910,kkh,0.432001,3
2020M2,0.548434,dsf,0.736445,5
2020M3,0.419726,gfhf,0.433392,6
2020M4,0.307432,ghf,0.389264,8
2020M5,0.001309,sdf,0.541600,2


### Import csv

In [3]:
from datetime import datetime
dtype = {
    "col3":object,
    "col5":int,
}
df = pd.read_csv(
    "example_data/csv_import_test.csv",
    sep=";",
    encoding="ISO-8859-1",
    dtype=dtype,
    decimal=",",
)
df.head()

,Unnamed: 0,col1,col2,col3,col4,col5
0,2020M1,0.10291012,0.302895796,kkh,0.432001348,3
1,2020M2,0.548434088,0.261697935,dsf,0.7364454,5
2,2020M3,0.41972556,0.565517074,gfhf,0.433392388,6
3,2020M4,0.307432482,0.923219617,ghf,0.389263708,8
4,2020M5,0.001309112,0.042402694,sdf,0.541600102,2


### Export Excel file
Styling: http://pbpython.com/improve-pandas-excel-output.html

#### Creates new/rewrites whole old book

In [4]:
# Example applies to Windows paths
df = pd.DataFrame(data={"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Age":[50, 25, 41], "Values":[3, 4, 8]})
writer = pd.ExcelWriter("example_data/excel_export_test.xlsx")
df.to_excel(writer, "Sheet1", header=True)
writer.save()

#### Only rewrites range in chosen sheet, not other sheets

Other sheets and data on selected sheet outside range are preserved.

In [5]:
from openpyxl import load_workbook
df = pd.DataFrame(
    data={"ID":[1, 2, 3, 4], "Names":["Kare", "John", "Max", "Mary"], "Age":[50, 25, 41,99], "Values":[3, 4, 8,4]},
    index = ["a", "b", "c", "d"]
)
book = load_workbook("example_data/excel_export_test.xlsx")
writer = pd.ExcelWriter("example_data/excel_export_test.xlsx", engine="openpyxl")
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
df.to_excel(writer, "Sheet1", header=True, index=False, startrow=0, startcol=6)
writer.save()

#### Rewrites chosen sheet but not other sheets

In [6]:
from openpyxl import load_workbook
df = pd.DataFrame(
    data={"ID":[1, 2, 3, 4], "Names":["Kare", "John", "Max", "Mary"], "Age":[50, 25, 41,99], "Values":[3, 4, 8,4]},
    index = ["a", "b", "c", "d"]
)
book = load_workbook("example_data/excel_export_test.xlsx")
if "sheetname" in book.sheetnames:
    del book["sheetname"]
    book.create_sheet("sheetname")
writer = pd.ExcelWriter("example_data/excel_export_test.xlsx", engine="openpyxl")
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
df.to_excel(writer, "sheetname", header=True, index=False, startrow=0, startcol=0)
writer.save()
writer.close()

## SQL

### Import SQL table

In [7]:
#import pyodbc
#cnxn = pyodbc.connect(
#    "Driver={SQL Server};"
#    "Server=server_name;"
#    "Database=dbo_name;"
#    "Trusted_Connection=yes;"
#)
#cursor = cnxn.cursor()
#sql = "SELECT TOP 1000 * FROM dbo.table"
#df = pd.read_sql_query(sql, cnxn)
#cnxn.close()

### Export data frame to SQL table (MS SQL Server)

In [8]:
#from sqlalchemy import create_engine
#import urllib
#import pyodbc
#quoted = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER={server_name};DATABASE={db_name}")
#engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(quoted))
#df.to_sql(name=table_name, schema=schema_name, index=False, con=engine)
#engine.dispose()

## Output Latex table

In [9]:
df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uu"])

# buf can be path, in which case a file is written
df_latex = df.to_latex(buf=None)

# From pandas 1.0.0 onward one can add caption via keyword
# In prior versions one needs to add it manually. After this
# file can be written as per usual
table_name = "tab:mytab"
caption = "This is caption."
df_latex_wcap = df_latex + "".join([
    r"\caption{\label",
    f"{{{table_name}}}",
    f"{caption}",
    r"}",
])

# For manual writing
# with open (path, "w") as f:
#    f.write(df_latex_wcap)

print(df_latex)
print(df_latex_wcap)

\begin{tabular}{lrrrr}
\toprule
{} &        ff &        ee &        tt &        uu \\
\midrule
0 &  0.031499 &  0.000376 &  0.380035 &  0.763132 \\
1 &  0.883413 &  0.320400 &  0.495849 &  0.050817 \\
2 &  0.182108 &  0.714365 &  0.030504 &  0.296191 \\
3 &  0.519098 &  0.480565 &  0.061987 &  0.952153 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrr}
\toprule
{} &        ff &        ee &        tt &        uu \\
\midrule
0 &  0.031499 &  0.000376 &  0.380035 &  0.763132 \\
1 &  0.883413 &  0.320400 &  0.495849 &  0.050817 \\
2 &  0.182108 &  0.714365 &  0.030504 &  0.296191 \\
3 &  0.519098 &  0.480565 &  0.061987 &  0.952153 \\
\bottomrule
\end{tabular}
\caption{\label{tab:mytab}This is caption.}


# Basic DataFrame operations

## DataFrame initialization

In [10]:
# Initialize DataFrame with dummy data and custom indices
df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uu"])
display(df)

,ff,ee,tt,uu
0,0.393557,0.481283,0.180818,0.653412
1,0.885576,0.747379,0.058779,0.733297
2,0.817712,0.239299,0.812765,0.412230
3,0.910970,0.381724,0.004019,0.437532


In [11]:
# Initialize DataFrame with custom data, with inferred data types
data = {"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Age":[50, 25, 41], "Values":[3, 4, 8]}
df = pd.DataFrame(data=data)

# Set one of the columns as index
df2 = df.set_index("ID")
 # Remove index name
df2.index.name = None

# Re-setting index
df3 = df2.copy()
df3.reset_index(inplace=True, drop=True) # drop = True; denies addition of old index to columns

# Force dtypes of columns upon initialization of the frame
data = {"Age":pd.Series([50, 25, 41], dtype=np.int64), "Values":pd.Series([3.5, 4.4, 8.0], dtype=np.float64)}
df4 = pd.DataFrame(data=data)

display_side_by_side(df, df2, df3, df4)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 
 
 1 
 2 
 John 
 25 
 4 
 
 
 2 
 3 
 Max 
 41 
 8 
 
 
 
 
 
 
 Names 
 Age 
 Values 
 
 
 
 
 1 
 Kate 
 50 
 3 
 
 
 2 
 John 
 25 
 4 
 
 
 3 
 Max 
 41 
 8 
 
 
 
 
 
 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 Kate 
 50 
 3 
 
 
 1 
 John 
 25 
 4 
 
 
 2 
 Max 
 41 
 8 
 
 
 
 
 
 
 Age 
 Values 
 
 
 
 
 0 
 50 
 3.5 
 
 
 1 
 25 
 4.4 
 
 
 2 
 41 
 8.0

## Modifying DataFrame

### Renaming columns

In [12]:
data = {"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Age":[50, 25, 41], "Values":[3, 4, 8]}
df = pd.DataFrame(data=data)
df2 = df.copy()
df.rename(columns = {"Names":"Name", "Age":"Alter"}, inplace=True)
display_side_by_side(df2, df)

,ID,Names,Age,Values
0,1,Kate,50,3
1,2,John,25,4
2,3,Max,41,8
,ID,Name,Alter,Values
0,1,Kate,50,3
1,2,John,25,4
2,3,Max,41,8


### Sort data frame by colum values

In [13]:
data = {"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Age":[50, 25, 41], "Values":[3, 4, 8]}
df = pd.DataFrame(data=data)
df2 = df.sort_values(["Age", "Names"], ascending =[True, False]).copy()
display_side_by_side(df, df2)

,ID,Names,Age,Values
0,1,Kate,50,3
1,2,John,25,4
2,3,Max,41,8
,ID,Names,Age,Values
1,2,John,25,4
2,3,Max,41,8
0,1,Kate,50,3


### Sort data frame by colums, custom order

In [14]:
data = {"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Age":[50, 25, 41], "Values":[3, 4, 8]}
df = pd.DataFrame(data=data)
df2 = df.copy()
df2.index = pd.Categorical(df2.index, [2, 0, 1])
df2 = df2.sort_index(axis=0)
display_side_by_side(df, df2)

,ID,Names,Age,Values
0,1,Kate,50,3
1,2,John,25,4
2,3,Max,41,8
,ID,Names,Age,Values
2,3,Max,41,8
0,1,Kate,50,3
1,2,John,25,4


### Change DataFrame column order

In [15]:
data = {"ID":[1, 2,], "Names":["John", "Max"], "Vals1":[50, 41], "Vals2":[4.0, 8.0], "Vals3":["4.4", "3.1"]}
df = pd.DataFrame(data=data)
df2 = df.iloc[:, ::-1].copy()
df3 = df[["Vals2", "Names", "Vals3", "Vals1"]].copy()
display_side_by_side(df, df2, df3)

ID 
 Names 
 Vals1 
 Vals2 
 Vals3 
 
 
 
 
 0 
 1 
 John 
 50 
 4.0 
 4.4 
 
 
 1 
 2 
 Max 
 41 
 8.0 
 3.1 
 
 
 
 
 
 
 Vals3 
 Vals2 
 Vals1 
 Names 
 ID 
 
 
 
 
 0 
 4.4 
 4.0 
 50 
 John 
 1 
 
 
 1 
 3.1 
 8.0 
 41 
 Max 
 2 
 
 
 
 
 
 
 Vals2 
 Names 
 Vals3 
 Vals1 
 
 
 
 
 0 
 4.0 
 John 
 4.4 
 50 
 
 
 1 
 8.0 
 Max 
 3.1 
 41

### Changing data types

In [16]:
# A type "O" just stands for "object" which in Pandas" world is a string (characters). 
data = {
    "ID":[1, 2, 3],
    "Names":["Kate", "John", "Max"],
    "Vals1":[50, 25, 41],
    "Vals2":[3.0, 4.0, 8.0],
    "Vals3":["3.0", "4.4", "3.1"],
    "Vals4":[3.0, np.nan, 3.1]
}
df = pd.DataFrame(data=data)
df2 = df.copy()

# Here we convert to numpy data types instead of native python types (float/int)
# Numpy data types are of fixed size but are more efficient
df2["Vals1"] = df2["Vals1"].astype("float64")
df2["Vals2"] = df2["Vals2"].astype("int64")
df2["Vals3"] = df2["Vals3"].astype("float64") # string to float, see also pd.to_numeric

# convert to strong but preserve NaNs
df2.loc[df2["Vals4"].notnull(), "Vals4"] = df2["Vals4"].astype(str)

# Check
print("Before Vals4 type was {}".format(df["Vals4"].dtype))
print("After Vals4 type was {}".format(df2["Vals4"].dtype))

print("This should be str: {}".format(type(df2.loc[0,"Vals4"])))
print("This should be float: {}".format(type(df2.loc[1,"Vals4"])))

display_side_by_side(df, df2)

Before Vals4 type was float64
After Vals4 type was object
This should be str: <class 'str'>
This should be float: <class 'float'>


,ID,Names,Vals1,Vals2,Vals3,Vals4
0,1,Kate,50,3.0,3.0,3.0
1,2,John,25,4.0,4.4,NaN
2,3,Max,41,8.0,3.1,3.1
,ID,Names,Vals1,Vals2,Vals3,Vals4
0,1,Kate,50.0,3,3.0,3.0
1,2,John,25.0,4,4.4,NaN
2,3,Max,41.0,8,3.1,3.1


### Add row to DataFrame

In [17]:
data={"ID":[1, 2], "Names":["John", "Max"],"Vals1":[50, 41], "Vals2":[4.0, 8.0], "Vals3":["4.4", "3.1"]}
df = pd.DataFrame(data=data)

df2 = pd.DataFrame({"ID":[4], "Names":["Janet"], "Vals1":[50], "Vals2":[12.0], "Vals3":["2.9"]})
df3 = df.append(df2, ignore_index=True)

display_side_by_side(df,df3)

,ID,Names,Vals1,Vals2,Vals3
0,1,John,50,4.0,4.4
1,2,Max,41,8.0,3.1
,ID,Names,Vals1,Vals2,Vals3
0,1,John,50,4.0,4.4
1,2,Max,41,8.0,3.1
2,4,Janet,50,12.0,2.9


### Add column to DataFrame

In [18]:
df = pd.DataFrame(data={"ID":[1, 2, 3], "Names":["John", "Max", "Jane"], "Vals1":[50, 41, 32]})
df2 = pd.DataFrame(data={"Vals2":[0.8, 0.6, 0.4]})
df3 = pd.concat([df, df2], axis=1)
display_side_by_side(df, df2, df3)

ID 
 Names 
 Vals1 
 
 
 
 
 0 
 1 
 John 
 50 
 
 
 1 
 2 
 Max 
 41 
 
 
 2 
 3 
 Jane 
 32 
 
 
 
 
 
 
 Vals2 
 
 
 
 
 0 
 0.8 
 
 
 1 
 0.6 
 
 
 2 
 0.4 
 
 
 
 
 
 
 ID 
 Names 
 Vals1 
 Vals2 
 
 
 
 
 0 
 1 
 John 
 50 
 0.8 
 
 
 1 
 2 
 Max 
 41 
 0.6 
 
 
 2 
 3 
 Jane 
 32 
 0.4

### Dropping from DataFrame

In [19]:
data = {"ID":[1, 2, 3, 4, 5, 6]
        ,"Names":["Kate", "John", "Max", "Mary", "Pete", "Miriam"]
        ,"Age":[50, 25, 41, 99, 54, 23]
        ,"Values":[3, 4, 8, 4, 3, 4]
        ,"col":[None]}
df_orig = pd.DataFrame(data=data, index=["a", "b", "c", "d", "e", "f"])
df = df_orig.copy()

# Drop columns by column label
df.drop(["Age", "Names"], axis=1, inplace=True)

# Drop column by fancy indexing
df.drop(df.columns[0], axis=1, inplace=True)

# Drop column by using del; is directly inplace!
del df["col"]

# Drop rows by index label
df.drop(["b"], axis=0 , inplace=True)

# Drop rows by index number
df.drop(df.index[[0]], axis=0, inplace=True)

# Drop rows where value in column does not fulfill condition
df = df[df["Values"] <= 5]

# Drop first n = 1 rows
df.drop(df.head(1).index,inplace=True)

# Drop last n = 1 rows
df.drop(df.tail(1).index,inplace=True) # drop last n rows

display_side_by_side(df_orig,df)

ID 
 Names 
 Age 
 Values 
 col 
 
 
 
 
 a 
 1 
 Kate 
 50 
 3 
 None 
 
 
 b 
 2 
 John 
 25 
 4 
 None 
 
 
 c 
 3 
 Max 
 41 
 8 
 None 
 
 
 d 
 4 
 Mary 
 99 
 4 
 None 
 
 
 e 
 5 
 Pete 
 54 
 3 
 None 
 
 
 f 
 6 
 Miriam 
 23 
 4 
 None 
 
 
 
 
 
 
 Values 
 
 
 
 
 e 
 3

In [20]:
# Drop columns except those we want to keep
data = {"ID":[1, 2, 3, 4], "Names":["Kate", "John", "Max", "Mary"], "Age":[50, 25, 41,99], "Values":[3, 4, 8,4], "col":[None]}
df = pd.DataFrame(data=data, index=["a", "b", "c", "d"])
columns_to_keep = ["Names", "Values"]
df2 = df[columns_to_keep].copy()
display_side_by_side(df, df2)

ID 
 Names 
 Age 
 Values 
 col 
 
 
 
 
 a 
 1 
 Kate 
 50 
 3 
 None 
 
 
 b 
 2 
 John 
 25 
 4 
 None 
 
 
 c 
 3 
 Max 
 41 
 8 
 None 
 
 
 d 
 4 
 Mary 
 99 
 4 
 None 
 
 
 
 
 
 
 Names 
 Values 
 
 
 
 
 a 
 Kate 
 3 
 
 
 b 
 John 
 4 
 
 
 c 
 Max 
 8 
 
 
 d 
 Mary 
 4

### Altering DataFrame values

In [21]:
df = pd.DataFrame(np.arange(12).reshape(6,2), columns=["ff", "er"])
df2 = df.copy()

# Apply change to whole column
df2["ff"] *= 0.8

# Conditionally modify certain cells
df2.loc[df2["er"] < 7, "er"] -= 2

# using np.where, create altered column
df2["altered_ff"]=np.where(df2["ff"] < 7, 1, 0)

display_side_by_side(df, df2)

ff 
 er 
 
 
 
 
 0 
 0 
 1 
 
 
 1 
 2 
 3 
 
 
 2 
 4 
 5 
 
 
 3 
 6 
 7 
 
 
 4 
 8 
 9 
 
 
 5 
 10 
 11 
 
 
 
 
 
 
 ff 
 er 
 altered_ff 
 
 
 
 
 0 
 0.0 
 -1 
 1 
 
 
 1 
 1.6 
 1 
 1 
 
 
 2 
 3.2 
 3 
 1 
 
 
 3 
 4.8 
 7 
 1 
 
 
 4 
 6.4 
 9 
 1 
 
 
 5 
 8.0 
 11 
 0

### Concatenate two columns into one

In [22]:
data = {
    "ID":[1, 2, 3],
    "Names":["Kate", "John", "Max"],
    "Occupation":["CEO", "Doctor", "Driver"],
    "Vals1":[50, 25, 41],
    "Vals2":[3.0, 4.0, 8.0]
}
df = pd.DataFrame(data=data)
df2 = df.copy()

df2["two_string_cols"] = df2["Names"] + "_" + df2["Occupation"]
df2["str_and_numeric"] = df2["Names"] + "_" + df2["Vals2"].map(str)
df2["format_numerics"] = df2["Names"] + "_" + df2["Vals2"].map("{:.0f}".format)

display_side_by_side(df, df2)

ID 
 Names 
 Occupation 
 Vals1 
 Vals2 
 
 
 
 
 0 
 1 
 Kate 
 CEO 
 50 
 3.0 
 
 
 1 
 2 
 John 
 Doctor 
 25 
 4.0 
 
 
 2 
 3 
 Max 
 Driver 
 41 
 8.0 
 
 
 
 
 
 
 ID 
 Names 
 Occupation 
 Vals1 
 Vals2 
 two_string_cols 
 str_and_numeric 
 format_numerics 
 
 
 
 
 0 
 1 
 Kate 
 CEO 
 50 
 3.0 
 Kate_CEO 
 Kate_3.0 
 Kate_3 
 
 
 1 
 2 
 John 
 Doctor 
 25 
 4.0 
 John_Doctor 
 John_4.0 
 John_4 
 
 
 2 
 3 
 Max 
 Driver 
 41 
 8.0 
 Max_Driver 
 Max_8.0 
 Max_8

### Calculating new column conditionally

In [23]:
data = {"ID":[1, 2, 3],"Names":["Kate", "John", "Max"],"val1":[50, 25, 41], "val2":[3.0, 0, 8.0]}
df = pd.DataFrame(data=data)
df2 = df.copy()
df3 = df.copy()

df2["val1/val2"]  = df2["val1"]/df2["val2"]
df3["val1/val2"]  = np.where(df2["val2"] != 0, df2["val1"]/df2["val2"], np.nan)
df2["val1"]/df2["val2"]

display_side_by_side(df, df2, df3)

ID 
 Names 
 val1 
 val2 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3.0 
 
 
 1 
 2 
 John 
 25 
 0.0 
 
 
 2 
 3 
 Max 
 41 
 8.0 
 
 
 
 
 
 
 ID 
 Names 
 val1 
 val2 
 val1/val2 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3.0 
 16.666667 
 
 
 1 
 2 
 John 
 25 
 0.0 
 inf 
 
 
 2 
 3 
 Max 
 41 
 8.0 
 5.125000 
 
 
 
 
 
 
 ID 
 Names 
 val1 
 val2 
 val1/val2 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3.0 
 16.666667 
 
 
 1 
 2 
 John 
 25 
 0.0 
 NaN 
 
 
 2 
 3 
 Max 
 41 
 8.0 
 5.125000

## Slicing data frame

### Basic sclicing
Using data[2] or data["label"] indexes the series itself, skipping the iloc/loc attributes. This is syntatic sugar, but it is safer to use iloc/loc attributes explicitly in order avoid confusion querying between index label and index position.

In [24]:
df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uuu"], index=["first", "second", "third", "fourth"])

# Selecting columns
# Using single square brackets selects a Series from DataFrame. 
# Using multiple square brackets selects a slice of data frame (possibly with muliple columns) 
df["ff"] # returns Series
df[["ff"]] # returns DataFrame with one column
df[["ff", "ee"]] # returns DataFrame with two columns
df.loc[:, ["ee", "tt"]]
df.iloc[:, 0] # by position in frame
df.iloc[:, 0:3] # first 3 columns

# Selecting rows
df.loc["first"] # by index
df.loc[~df.index.isin(["first"])] # rows except index "first"
df.loc[["first", "fourth"]] # by index multiple rows
df[0:3]
df.iloc[0:2] # by position in frame

# Select ith row of column "ee"
df.iloc[2:4,list(df.columns).index("ee")]

display_side_by_side(
    df,
    df.loc[:,["ee","tt"]],
    df.iloc[0:2],
    df.loc[~df.index.isin(["first"])], df.iloc[:,0:3],
    pd.DataFrame(df.iloc[2:4,list(df.columns).index("ee")])
)

ff 
 ee 
 tt 
 uuu 
 
 
 
 
 first 
 0.090478 
 0.350420 
 0.877982 
 0.561611 
 
 
 second 
 0.532830 
 0.765480 
 0.586204 
 0.863395 
 
 
 third 
 0.845539 
 0.901735 
 0.408022 
 0.507302 
 
 
 fourth 
 0.560988 
 0.153830 
 0.217356 
 0.388269 
 
 
 
 
 
 
 ee 
 tt 
 
 
 
 
 first 
 0.350420 
 0.877982 
 
 
 second 
 0.765480 
 0.586204 
 
 
 third 
 0.901735 
 0.408022 
 
 
 fourth 
 0.153830 
 0.217356 
 
 
 
 
 
 
 ff 
 ee 
 tt 
 uuu 
 
 
 
 
 first 
 0.090478 
 0.35042 
 0.877982 
 0.561611 
 
 
 second 
 0.532830 
 0.76548 
 0.586204 
 0.863395 
 
 
 
 
 
 
 ff 
 ee 
 tt 
 uuu 
 
 
 
 
 second 
 0.532830 
 0.765480 
 0.586204 
 0.863395 
 
 
 third 
 0.845539 
 0.901735 
 0.408022 
 0.507302 
 
 
 fourth 
 0.560988 
 0.153830 
 0.217356 
 0.388269 
 
 
 
 
 
 
 ff 
 ee 
 tt 
 
 
 
 
 first 
 0.090478 
 0.350420 
 0.877982 
 
 
 second 
 0.532830 
 0.765480 
 0.586204 
 
 
 third 
 0.845539 
 0.901735 
 0.408022 
 
 
 fourth 
 0.560988 
 0.153830 
 0.217356 
 
 
 
 
 
 
 ee 
 
 
 
 
 third 
 0.901735 
 
 
 fourth 
 0.153830

### Conditional slicing/indexing (boolean masking)

In [25]:
data = {"ID":[1, 2, 3, 4], "Names":["Kate", "John", "Max", "Mary"], "Age":[50, 25, 41,99], "Values":[3, 4, 8,4]}
df = pd.DataFrame(data=data, index=["a", "b", "c", "d"])

# Get row slice fulfilling a condition along one column, return only
# certain columns
df2 = pd.DataFrame(df.loc[df["Values"] < 5,  ["Names", "Values"]].copy())

# Same as above but multiple conditions, return all columns tha match
df3 = df.loc[(df["Values"] < 5 ) & (df["Names"] == "John"), ].copy()

# Conditional indexing by preserving original dimensions
df4 = df.where(df["Values"] < 5).copy()

display_side_by_side(df, df2, df3, df4)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 a 
 1 
 Kate 
 50 
 3 
 
 
 b 
 2 
 John 
 25 
 4 
 
 
 c 
 3 
 Max 
 41 
 8 
 
 
 d 
 4 
 Mary 
 99 
 4 
 
 
 
 
 
 
 Names 
 Values 
 
 
 
 
 a 
 Kate 
 3 
 
 
 b 
 John 
 4 
 
 
 d 
 Mary 
 4 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 
 
 
 
 b 
 2 
 John 
 25 
 4 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 
 
 
 
 a 
 1.0 
 Kate 
 50.0 
 3.0 
 
 
 b 
 2.0 
 John 
 25.0 
 4.0 
 
 
 c 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 d 
 4.0 
 Mary 
 99.0 
 4.0

### Slicing based on partial string match in column

In [26]:
data = {"ID":[1, 2, 3, 4], "Names":["Kare", "John", "Max", "Mary"], "Age":[50, 25, 41,99], "Values":[3, 4, 8,4]}
df = pd.DataFrame(data=data, index=["a", "b", "c", "d"])
df2 = df.copy()

df2 = df2[df2["Names"].str.contains("ar")]

display_side_by_side(df, df2)

,ID,Names,Age,Values
a,1,Kare,50,3
b,2,John,25,4
c,3,Max,41,8
d,4,Mary,99,4
,ID,Names,Age,Values
a,1,Kare,50,3
d,4,Mary,99,4


### Select rows by n largest/smalles in one column
If the nth largest/smallest is a duplicate value, will return al of those. The way here is most deninitely not the most elegant solution...

In [27]:
data = {"ID":[1, 2, 3, 4, 5], "Names":["Kare", "John", "Max", "Mary", "Santa"], "Age":[50, 25, 41,99, 23], "Values":[3, 4, 8, 4, 2]}
df = pd.DataFrame(data=data)

df2 = df[df["Values"] >= df["Values"].nlargest(2).values[-1]].copy()
df3 = df[df["Values"] <= df["Values"].nsmallest(3).values[-1]].copy()

display_side_by_side(df, df2, df3)

,ID,Names,Age,Values
0,1,Kare,50,3
1,2,John,25,4
2,3,Max,41,8
3,4,Mary,99,4
4,5,Santa,23,2
,ID,Names,Age,Values
1,2,John,25,4
2,3,Max,41,8
3,4,Mary,99,4
,ID,Names,Age,Values


## Iterating over DataFrame

In [28]:
data = {"Names":["Kate", "John", "Max"], "Age":[50, 25, 41], "Weight":[62, 76, 98]}
df = pd.DataFrame(data=data)

# "Right" way with df.iterrows 
for index, row in df.iterrows():
    print(index, row["Age"])

print("-"*30)    

# "Typical" integer looping over rows
for counter in range(0,len(df)):
    print(df.iloc[counter,0])   

0 50
1 25
2 41
------------------------------
Kate
John
Max


In [29]:
# Set values during row iteration
data = {"Names":["Kate", "John", "Max"], "Age":[50, 25, 41], "Weight":[62, 76, 98]}
df = pd.DataFrame(data=data)
df2 = df.copy()

for index, row in df2.iterrows():
    if row["Names"] == "John":
        df2.at[index, "Weight"] = 100
display_side_by_side(df, df2)    

,Names,Age,Weight
0,Kate,50,62
1,John,25,76
2,Max,41,98
,Names,Age,Weight
0,Kate,50,62
1,John,25,100
2,Max,41,98


## About chain indexing

In [30]:
# A rule of thub is that any time you see back-to-back square brackets,
# ][, you"re asking for trouble. Replace that with a .loc[] or .iloc[]
# and you"ll be set. Let"s see an example

df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uuu"], index=["first", "second", "third", "fourth"])
display(df)
print(df["ee"]["second"]) # bad practice
print(df.loc["second", "ee"]) # good practice

,ff,ee,tt,uuu
first,0.679840,0.165465,0.483168,0.103914
second,0.960658,0.079341,0.848851,0.509292
third,0.034396,0.615180,0.487361,0.508869
fourth,0.660448,0.259019,0.234198,0.274732


0.07934107958399716
0.07934107958399716


In [31]:
# Another example
df = pd.DataFrame()
df["x"] = np.arange(10,14)
df["value"] = np.arange(200,204)
display(df)

print(df[ df["x"] == 10]["value"]) # bad practice
print("-"*30)
print(df.loc[df["x"] == 10, "value"]) # good practice

# The bad practice option would"t even work in the case
# we want to assign a new value for the returned obseravtions
#df[ df["x"]== 10 ]["value"] = 1000 # this throws error!
print("-"*30)
df.loc[df["x"] == 10, "value"] = 1000
display(df)

,x,value
0,10,200
1,11,201
2,12,202
3,13,203


0    200
Name: value, dtype: int32
------------------------------
0    200
Name: value, dtype: int32
------------------------------


,x,value
0,10,1000
1,11,201
2,12,202
3,13,203


## Method chaining

In [32]:
# "Pandorable" way is to chain methods operating on DataFrame:
data = {"Names":["Kate", "John", "Max"],"Age":[50, 25, 41], "Weight":[62, 76, 98]}
df = pd.DataFrame(data=data)

df2 = df.copy()
# Not-so-good practice: two lines of code
df2 = df2.drop(df2[df2["Age"] == 25].index)
df2.rename(columns={"Weight":"Weight (kg)"}, inplace=True)

df3 = df.copy()
# Better practice - on line of code
df3 = df3.drop(df3[df3["Age"] == 25].index).rename(columns={"Weight":"Weight (kg)"}).copy()

display_side_by_side(df, df2, df3)

,Names,Age,Weight
0,Kate,50,62
1,John,25,76
2,Max,41,98
,Names,Age,Weight (kg)
0,Kate,50,62
2,Max,41,98
,Names,Age,Weight (kg)
0,Kate,50,62
2,Max,41,98


# Data types and scales

This section is till incomplete!

In general, there are 4 types of data scales
<ol>
  <li>Ratio scale, e.g. height</li>
  <li>Inteval scale, e.g. temperature</li>
  <li>Ordinal scale, e.g. grading scale</li>
  <li>Nominal scale, e.g. school names</li>
</ol>

In [33]:
from pandas.api.types import CategoricalDtype
df = pd.DataFrame(["E", "M", "L", "M", "I", "A", "B", "B", "M"], columns=["Grades"])

cats = ["I", "A", "B", "C", "M", "E", "L"]
cat_type = CategoricalDtype(categories=cats, ordered=True)
df["Grades"] = df["Grades"].astype(cat_type)

df2 = df.sort_values(["Grades"], ascending=False).copy()
df3 = df[df["Grades"] >= "M"].copy()

display_side_by_side(df, df2, df3)

,Grades
0,E
1,M
2,L
3,M
4,I
5,A
6,B
7,B
8,M
,Grades


# Hierarchical indexing (MultiIndex)

## Initialize DataFrame with hierarchical indexing

In [34]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2"], ["ff","gg","ff"]]
)
display_side_by_side(df)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

## Modifying MultiIndex DataFrame

### Add second index to existing DataFrame

In [35]:
df = pd.DataFrame(data=np.arange(12).reshape((4,3)), columns=["col1","col1","col2"])
df2 = df.copy()
df2.set_index([df2.index, "col2"], inplace=True)
df2.index.names = ["Index1", "Index2"]
display_side_by_side(df, df2)

,col1,col1,col2
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
,,col1,col1
Index1,Index2,,
0,2,0,1
1,5,3,4
2,8,6,7
3,11,9,10


### Add new column to existing MultiIndex DataFrame

In [36]:
df = pd.DataFrame(
    data=np.arange(16).reshape((4,4)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2","col2"], ["ff","gg","ff","gg"]]
)

# Add new column to MultiIndex frame
df2 = df.copy()
df2["col2", "dd"] = [0, 1, 2, 4]
display_side_by_side(df, df2)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 3 
 
 
 2 
 4 
 5 
 6 
 7 
 
 
 b 
 1 
 8 
 9 
 10 
 11 
 
 
 2 
 12 
 13 
 14 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 dd 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 3 
 0 
 
 
 2 
 4 
 5 
 6 
 7 
 1 
 
 
 b 
 1 
 8 
 9 
 10 
 11 
 2 
 
 
 2 
 12 
 13 
 14 
 15 
 4

### Add new column level to existing MultiIndex DataFrame

In [37]:
df = pd.DataFrame(
    data=np.arange(16).reshape((4,4)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2","col2"], ["ff","gg","ff","gg"]]
)

df2 = df.copy()
df2.columns = pd.MultiIndex.from_tuples([(c1, "new_level", c2) for c1, c2 in df2.columns])

display_side_by_side(df, df2)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 3 
 
 
 2 
 4 
 5 
 6 
 7 
 
 
 b 
 1 
 8 
 9 
 10 
 11 
 
 
 2 
 12 
 13 
 14 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 new_level 
 new_level 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 3 
 
 
 2 
 4 
 5 
 6 
 7 
 
 
 b 
 1 
 8 
 9 
 10 
 11 
 
 
 2 
 12 
 13 
 14 
 15

### Get different MultiIndex levels

In [38]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2"], ["ff","gg","ff"]]
)

print(df.columns.get_level_values(0))
print(df.columns.get_level_values(1))
print(df.index.get_level_values(0))
print(df.index.get_level_values(1))
print(df.index.levels[1])
print(df.columns.levels[0])
display(df)

Index(['col1', 'col1', 'col2'], dtype='object')
Index(['ff', 'gg', 'ff'], dtype='object')
Index(['a', 'a', 'b', 'b'], dtype='object')
Int64Index([1, 2, 1, 2], dtype='int64')
Int64Index([1, 2], dtype='int64')
Index(['col1', 'col2'], dtype='object')


col1     col2
      ff  gg   ff
a 1    0   1    2
  2    3   4    5
b 1    6   7    8
  2    9  10   11

### Set MultiIndex column names anew

In [39]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a", "a", "b", "b"], [1, 2]*2],
    columns=[["col1", "col1", "col2"], ["ff", "gg", "ff"]]
)
df2 = df.copy()
df2 = df.copy()
df3 = df.copy()

# Using MultiIndex.from_product
new_cols = pd.MultiIndex.from_product([["col1new", "col2new"], ["hh", "jj"]])
new_cols = new_cols[:-1]
df2.columns = new_cols

# Using MultiIndex.from_tuples
tuples = list(zip(df3.columns.get_level_values(0), ["aa", "ss", "rrr"]))
df3.columns = pd.MultiIndex.from_tuples(tuples)

display_side_by_side(df, df2, df3)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 
 col1new 
 col2new 
 
 
 
 
 hh 
 jj 
 hh 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 aa 
 ss 
 rrr 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

### Rename MultiIndex columns

In [40]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a", "a", "b", "b"], [1, 2]*2],
    columns=[["col1", "col1", "col2"], ["ff", "gg", "ff"]]
)
# Given xth level column
df2 = df.rename(columns={"col1": "sss"}, level=0)

# Second level column from given 1st level column
# This is quite hacky...
df3 = df.copy()
new_2nd_lvl_vals = [x if (y != "col2" or x != "ff") else "new" for y, x in zip(df.columns.get_level_values(0), df.columns.get_level_values(1))]
tuples = list(zip(df.columns.get_level_values(0), new_2nd_lvl_vals))
df3.columns = pd.MultiIndex.from_tuples(tuples)

display_side_by_side(df, df2, df3)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 
 sss 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 new 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

### Re-order specific multi-index level 

In [41]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2"], ["ff","gg","ff"]]
)
display_side_by_side(
    df,
    df.reindex(["b", "a"], level=0, axis=0),
    df.reindex(["gg", "ff"], level=1, axis=1),
)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 gg 
 ff 
 ff 
 
 
 
 
 a 
 1 
 1 
 0 
 2 
 
 
 2 
 4 
 3 
 5 
 
 
 b 
 1 
 7 
 6 
 8 
 
 
 2 
 10 
 9 
 11

### Swap levels in multi-index/multi-column

In [42]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2"], ["ff","gg","ff"]]
)
df.index.names = ["idx_lvl1", "idx_lvl2"]
df.columns.names = ["col_lvl1", "col_lvl2"]

display_side_by_side(
    df,
    df.swaplevel(i="idx_lvl2", j="idx_lvl1", axis=0),
    df.swaplevel(i="col_lvl2", j="col_lvl1", axis=1),
)

col_lvl1 
 col1 
 col2 
 
 
 
 col_lvl2 
 ff 
 gg 
 ff 
 
 
 idx_lvl1 
 idx_lvl2 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 col_lvl1 
 col1 
 col2 
 
 
 
 col_lvl2 
 ff 
 gg 
 ff 
 
 
 idx_lvl2 
 idx_lvl1 
 
 
 
 
 
 
 
 1 
 a 
 0 
 1 
 2 
 
 
 2 
 a 
 3 
 4 
 5 
 
 
 1 
 b 
 6 
 7 
 8 
 
 
 2 
 b 
 9 
 10 
 11 
 
 
 
 
 
 
 col_lvl2 
 ff 
 gg 
 ff 
 
 
 
 col_lvl1 
 col1 
 col1 
 col2 
 
 
 idx_lvl1 
 idx_lvl2 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11

### Swap multi-index level to multi-column level

In [43]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2"], ["ff","gg","ff"]]
)
df.index.names = ["idx_lvl1", "idx_lvl2"]
df.columns.names = ["col_lvl1", "col_lvl2"]

display_side_by_side(
    df,
    df.unstack(level="idx_lvl2"),
)

col_lvl1 
 col1 
 col2 
 
 
 
 col_lvl2 
 ff 
 gg 
 ff 
 
 
 idx_lvl1 
 idx_lvl2 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 col_lvl1 
 col1 
 col2 
 
 
 col_lvl2 
 ff 
 gg 
 ff 
 
 
 idx_lvl2 
 1 
 2 
 1 
 2 
 1 
 2 
 
 
 idx_lvl1 
 
 
 
 
 
 
 
 
 
 
 a 
 0 
 3 
 1 
 4 
 2 
 5 
 
 
 b 
 6 
 9 
 7 
 10 
 8 
 11

### Swap multi-coluns level to multi-index level

In [44]:
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2"], ["ff","gg","ff"]]
)
df.index.names = ["idx_lvl1", "idx_lvl2"]
df.columns.names = ["col_lvl1", "col_lvl2"]

display_side_by_side(
    df,
    df.stack(level="col_lvl2"),
)

col_lvl1 
 col1 
 col2 
 
 
 
 col_lvl2 
 ff 
 gg 
 ff 
 
 
 idx_lvl1 
 idx_lvl2 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 
 col_lvl1 
 col1 
 col2 
 
 
 idx_lvl1 
 idx_lvl2 
 col_lvl2 
 
 
 
 
 
 
 a 
 1 
 ff 
 0 
 2.0 
 
 
 gg 
 1 
 NaN 
 
 
 2 
 ff 
 3 
 5.0 
 
 
 gg 
 4 
 NaN 
 
 
 b 
 1 
 ff 
 6 
 8.0 
 
 
 gg 
 7 
 NaN 
 
 
 2 
 ff 
 9 
 11.0 
 
 
 gg 
 10 
 NaN

## Quering MultiIndex DataFrames

In [45]:
df = pd.DataFrame(
    data=np.arange(16).reshape((4,4)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2","col2"], ["ff","gg","ff","gg"]]
)

# Get observations from certain column level
df2 = df.iloc[:, df.columns.get_level_values(1) == "gg"].copy()
df2b = df.iloc[:, df.columns.get_level_values(1).isin(["gg","ee"])].copy()

# Get observations from certain index level
df3 = df.iloc[df.index.get_level_values(1) == 2, :].copy()

# Get observations from certain column levels (multi-condition)
df5 = df.loc(axis=1)[["col1"],:].copy()
df4 = df.iloc[:, (df.columns.get_level_values(0) == "col1") & (df.columns.get_level_values(1) == "gg")].copy()

display_side_by_side(df, df2, df2b, df3, df4, df5)

col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 3 
 
 
 2 
 4 
 5 
 6 
 7 
 
 
 b 
 1 
 8 
 9 
 10 
 11 
 
 
 2 
 12 
 13 
 14 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 gg 
 gg 
 
 
 
 
 a 
 1 
 1 
 3 
 
 
 2 
 5 
 7 
 
 
 b 
 1 
 9 
 11 
 
 
 2 
 13 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 gg 
 gg 
 
 
 
 
 a 
 1 
 1 
 3 
 
 
 2 
 5 
 7 
 
 
 b 
 1 
 9 
 11 
 
 
 2 
 13 
 15 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 gg 
 
 
 
 
 a 
 2 
 4 
 5 
 6 
 7 
 
 
 b 
 2 
 12 
 13 
 14 
 15 
 
 
 
 
 
 
 
 col1 
 
 
 
 
 gg 
 
 
 
 
 a 
 1 
 1 
 
 
 2 
 5 
 
 
 b 
 1 
 9 
 
 
 2 
 13 
 
 
 
 
 
 
 
 col1 
 
 
 
 
 ff 
 gg 
 
 
 
 
 a 
 1 
 0 
 1 
 
 
 2 
 4 
 5 
 
 
 b 
 1 
 8 
 9 
 
 
 2 
 12 
 13

# Data wrangling tricks

## Mappings for column values

### One-to-one mappings

#### Map from dict

In [46]:
data = {"ID":[1, 2, 3,4],"Names":["Kate", "John", "Max","Kate"],"Age":[50, 25, 41,89], "Values":[3, 4, 8, 12]}
df = pd.DataFrame(data=data)
df2 = df.copy()

# Mapping from user defined dict
age_mapping = {25:0, 41:1, 50:2, 89:3}
df2["AgeOrder"] = df2["Age"].map(age_mapping)
display_side_by_side(df, df2)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 
 
 1 
 2 
 John 
 25 
 4 
 
 
 2 
 3 
 Max 
 41 
 8 
 
 
 3 
 4 
 Kate 
 89 
 12 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 AgeOrder 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 2 
 
 
 1 
 2 
 John 
 25 
 4 
 0 
 
 
 2 
 3 
 Max 
 41 
 8 
 1 
 
 
 3 
 4 
 Kate 
 89 
 12 
 3

#### Assign unique identifier to column values

In [47]:
data = {"ID":[1, 2, 3,4], "Names":["Kate", "John", "Max","Kate"], "Age":[50, 25, 41,89], "Values":[3, 4, 8, 12]}
df = pd.DataFrame(data=data)
df2 = df.copy()

# Assign unique identifier to values in column (Names -> NameID)
d = {key: value for (key, value) in zip(df["Names"].unique(),range(len(df["Names"].unique())))}
df2["NameID"] = df2["Names"].map(d)
display_side_by_side(df, df2)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 
 
 1 
 2 
 John 
 25 
 4 
 
 
 2 
 3 
 Max 
 41 
 8 
 
 
 3 
 4 
 Kate 
 89 
 12 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 NameID 
 
 
 
 
 0 
 1 
 Kate 
 50 
 3 
 0 
 
 
 1 
 2 
 John 
 25 
 4 
 1 
 
 
 2 
 3 
 Max 
 41 
 8 
 2 
 
 
 3 
 4 
 Kate 
 89 
 12 
 0

#### Map column string values to lowercase

In [48]:
data = {"ID":[1, 2, 3,4], "Names":["Kate", "JoHn", "MA4", "KATE"], "Values":[3, 4, 8, 12]}
df = pd.DataFrame(data=data)
df2 = df.copy()
df2["Names"] = df2["Names"].map(lambda x: x if type(x)!=str else x.lower())

display_side_by_side(df, df2)

,ID,Names,Values
0,1,Kate,3
1,2,JoHn,4
2,3,MA4,8
3,4,KATE,12
,ID,Names,Values
0,1,kate,3
1,2,john,4
2,3,ma4,8
3,4,kate,12


#### Using regex with map

In [49]:
import re
data = {"ID":[1, 2, 3,4], "Names":["Kate", "John", "Max","Kate"], "Values":["foff", "ss-d", "8", "-rrgr"]}
df = pd.DataFrame(data=data)
df2 = df.copy()

# also includes the delimiter!
regex2 = re.compile("(?:-).*")
df2["Values"] = df2["Values"].replace(to_replace=regex2, value="REPLACED")

display_side_by_side(df, df2)

,ID,Names,Values
0,1,Kate,foff
1,2,John,ss-d
2,3,Max,8
3,4,Kate,-rrgr
,ID,Names,Values
0,1,Kate,foff
1,2,John,ssREPLACED
2,3,Max,8
3,4,Kate,REPLACED


### Map values to ranges/groups

In [50]:
data = {"ID":[1, 2, 3,4], "Names":["Kate", "John", "Max","Kate"], "Age":[39, 18, 25,75], "Values":[3, 4, 8, 12]}
df = pd.DataFrame(data=data)
df2 = df.copy()

# Even-sized bins 
df2["ValGroup1"] = pd.cut(df2["Values"] , 4, right=True) # 4 bins, includes rightmost edge
                    
# Relative-sized bins 
df2["ValGroup2"] = pd.qcut(df2["Values"], 4)

# Custom bin sizes and labels
df2["AgeGroup"] = pd.cut(df2["Age"], [0, 20, 40, 60, 80, np.inf], labels=[1, 2, 3, 4, 5], right=True)

display_side_by_side(df, df2)

ID 
 Names 
 Age 
 Values 
 
 
 
 
 0 
 1 
 Kate 
 39 
 3 
 
 
 1 
 2 
 John 
 18 
 4 
 
 
 2 
 3 
 Max 
 25 
 8 
 
 
 3 
 4 
 Kate 
 75 
 12 
 
 
 
 
 
 
 ID 
 Names 
 Age 
 Values 
 ValGroup1 
 ValGroup2 
 AgeGroup 
 
 
 
 
 0 
 1 
 Kate 
 39 
 3 
 (2.991, 5.25] 
 (2.999, 3.75] 
 2 
 
 
 1 
 2 
 John 
 18 
 4 
 (2.991, 5.25] 
 (3.75, 6.0] 
 1 
 
 
 2 
 3 
 Max 
 25 
 8 
 (7.5, 9.75] 
 (6.0, 9.0] 
 2 
 
 
 3 
 4 
 Kate 
 75 
 12 
 (9.75, 12.0] 
 (9.0, 12.0] 
 4

### Variable to dummy columns

Here type Categorical but applies to other types as well.

In [51]:
df = pd.DataFrame({"Category":["Type1", "Type2", "Type1", "Type3"], "Value":np.random.rand(4)})

df2 = df.copy()
cats = ["Type1", "Type2", "Type3"]
cat_type = CategoricalDtype(categories=cats, ordered=True)
df["Category"] = df["Category"].astype(cat_type)
df2 = pd.get_dummies(df2, prefix=["Category"])

display_side_by_side(df, df2)

Category 
 Value 
 
 
 
 
 0 
 Type1 
 0.901253 
 
 
 1 
 Type2 
 0.122458 
 
 
 2 
 Type1 
 0.225422 
 
 
 3 
 Type3 
 0.777161 
 
 
 
 
 
 
 Value 
 Category_Type1 
 Category_Type2 
 Category_Type3 
 
 
 
 
 0 
 0.901253 
 1 
 0 
 0 
 
 
 1 
 0.122458 
 0 
 1 
 0 
 
 
 2 
 0.225422 
 1 
 0 
 0 
 
 
 3 
 0.777161 
 0 
 0 
 1

### Dummy columns back to index variable

In [52]:
df = pd.DataFrame({"Category":["Type1", "Type2", "Type1", "Type3"], "Value":np.random.rand(4)})
df = pd.get_dummies(df, prefix=["Category"])
df["Category_reversed"] = df[["Category_Type1", "Category_Type2", "Category_Type3"]].idxmax(axis=1)
display(df)

,Value,Category_Type1,Category_Type2,Category_Type3,Category_reversed
0,0.516585,1,0,0,Category_Type1
1,0.803643,0,1,0,Category_Type2
2,0.597174,1,0,0,Category_Type1
3,0.270524,0,0,1,Category_Type3


## Change between long/wide format

### Wide to long

In [53]:
# Create data
df = pd.DataFrame(data=np.arange(9).reshape((3,3)), columns=[["col1","col2","col3"]])

df2 = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2"], ["ff","gg","ff"]]
)

# Simple melt
df3 = pd.melt(df).copy()
df4 = pd.melt(df2).copy()

display_side_by_side(df,df3,df2,df4)

col1 
 col2 
 col3 
 
 
 
 
 0 
 0 
 1 
 2 
 
 
 1 
 3 
 4 
 5 
 
 
 2 
 6 
 7 
 8 
 
 
 
 
 
 
 None 
 value 
 
 
 
 
 0 
 col1 
 0 
 
 
 1 
 col1 
 3 
 
 
 2 
 col1 
 6 
 
 
 3 
 col2 
 1 
 
 
 4 
 col2 
 4 
 
 
 5 
 col2 
 7 
 
 
 6 
 col3 
 2 
 
 
 7 
 col3 
 5 
 
 
 8 
 col3 
 8 
 
 
 
 
 
 
 
 col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 variable_0 
 variable_1 
 value 
 
 
 
 
 0 
 col1 
 ff 
 0 
 
 
 1 
 col1 
 ff 
 3 
 
 
 2 
 col1 
 ff 
 6 
 
 
 3 
 col1 
 ff 
 9 
 
 
 4 
 col1 
 gg 
 1 
 
 
 5 
 col1 
 gg 
 4 
 
 
 6 
 col1 
 gg 
 7 
 
 
 7 
 col1 
 gg 
 10 
 
 
 8 
 col2 
 ff 
 2 
 
 
 9 
 col2 
 ff 
 5 
 
 
 10 
 col2 
 ff 
 8 
 
 
 11 
 col2 
 ff 
 11

### Long to wide

In [54]:
df = pd.DataFrame({
    "name1":["A","A","B","B"],
    "name2":["A","B","A","B"],
    "name3":["r","t","r","b"],
    "val":[5,9,7,3]
})

df2 = df.pivot_table(values="val", index=["name1","name3"], columns="name2").reset_index()
df2.columns.name = None

display_side_by_side(df, df2)

,name1,name2,name3,val
0,A,A,r,5
1,A,B,t,9
2,B,A,r,7
3,B,B,b,3
,name1,name3,A,B
0,A,r,5.0,NaN
1,A,t,NaN,9.0
2,B,b,NaN,3.0
3,B,r,7.0,NaN


In [55]:
# Pivot_table might not work with non-numeric columns
# In this case try the following (does however not work
# with duplicate indices

df = pd.DataFrame({
    "index" : range(8),
    "variable1":["A", "A", "B", "B", "A", "B", "B", "A"],
    "variable2":["a", "b", "a", "b", "a", "b", "a", "b"],
    "result":["on", "off", "off", "on", "on", "off", "off", "on"]
})

df1 = df.copy()
unstack_cols = ["variable1", "variable2"]
df1 = df.set_index(["index"] + unstack_cols).unstack(level=unstack_cols)

display_side_by_side(df,df1)

index 
 variable1 
 variable2 
 result 
 
 
 
 
 0 
 0 
 A 
 a 
 on 
 
 
 1 
 1 
 A 
 b 
 off 
 
 
 2 
 2 
 B 
 a 
 off 
 
 
 3 
 3 
 B 
 b 
 on 
 
 
 4 
 4 
 A 
 a 
 on 
 
 
 5 
 5 
 B 
 b 
 off 
 
 
 6 
 6 
 B 
 a 
 off 
 
 
 7 
 7 
 A 
 b 
 on 
 
 
 
 
 
 
 result 
 
 
 variable1 
 A 
 B 
 
 
 variable2 
 a 
 b 
 a 
 b 
 
 
 index 
 
 
 
 
 
 
 
 
 0 
 on 
 NaN 
 NaN 
 NaN 
 
 
 1 
 NaN 
 off 
 NaN 
 NaN 
 
 
 2 
 NaN 
 NaN 
 off 
 NaN 
 
 
 3 
 NaN 
 NaN 
 NaN 
 on 
 
 
 4 
 on 
 NaN 
 NaN 
 NaN 
 
 
 5 
 NaN 
 NaN 
 NaN 
 off 
 
 
 6 
 NaN 
 NaN 
 off 
 NaN 
 
 
 7 
 NaN 
 on 
 NaN 
 NaN

In [56]:
# Long to wide with time series type of data
data = {
    "ID":[1, 1, 2, 2, 1, 1, 2, 2],
    "Date":["2007-01-01", "2007-01-01", "2007-01-01", "2007-01-01", "2007-01-02","2007-01-02", "2007-01-02", "2007-01-02"],
    "Type":["type1", "type2", "type1", "type2", "type1", "type2", "type1", "type2"],
    "Values":np.arange(8)
}

# Using pivot_table
df = pd.DataFrame(data=data)
df.loc[4, "Values"] = np.nan
df4 = df.pivot_table(values="Values", index=["Date"], columns=["ID", "Type"])
df4.columns.names = [None,None]
df4.index.name = None
new_col_names = ["_".join((str(b), str(a))) for a, b in df4.columns.to_numpy()] # flatten columns
df4.columns = new_col_names

# Using unstack; if data contains NAs, these are converted to 0 which is NOT PREFERABLE!
df2 = df.groupby(["Date","ID","Type"]).sum().unstack("ID").copy()
new_col_names = ["_".join((str(b), a)) for a, b in df2.columns.to_numpy()] # flatten columns
df2.columns = new_col_names

df3 = df.groupby(["Date","ID","Type"]).sum().unstack(["ID","Type"]).copy()
new_col_names = ["_".join((c, str(b))) for a, b, c in df3.columns.to_numpy()]
df3.columns = new_col_names

display_side_by_side(df, df4, df3)

,ID,Date,Type,Values
0,1,2007-01-01,type1,0.0
1,1,2007-01-01,type2,1.0
2,2,2007-01-01,type1,2.0
3,2,2007-01-01,type2,3.0
4,1,2007-01-02,type1,NaN
5,1,2007-01-02,type2,5.0
6,2,2007-01-02,type1,6.0
7,2,2007-01-02,type2,7.0
,type1_1,type2_1,type1_2,type2_2
2007-01-01,0.0,1.0,2.0,3.0


## Database-like joins

In [57]:
# Create data
df = pd.DataFrame(data=np.arange(12).reshape((3, 4)), columns=["col1", "col2", "col3", "col4"])
df2 = pd.DataFrame({"key":[0, 1, 2], "vals":[4, 6, 8]})
display_side_by_side(df, df2)

col1 
 col2 
 col3 
 col4 
 
 
 
 
 0 
 0 
 1 
 2 
 3 
 
 
 1 
 4 
 5 
 6 
 7 
 
 
 2 
 8 
 9 
 10 
 11 
 
 
 
 
 
 
 key 
 vals 
 
 
 
 
 0 
 0 
 4 
 
 
 1 
 1 
 6 
 
 
 2 
 2 
 8

In [58]:
# Different merges
df3 = pd.merge(df, df2, left_index=True, right_on="key", how="left")
df4 = pd.merge(df, df2, left_on="col2" , right_on="key", how="left")
df5 = pd.merge(df, df2, left_on="col2" , right_on="key", how="right")
df6 = pd.merge(df, df2, left_on="col2" , right_on="key", how="inner")
df7 = pd.merge(df, df2, left_on="col2" , right_on="key", how="outer")

# Right and left joins are basically the same, just different way around
# Compare df8 to df5
df8 = pd.merge(df2, df, left_on="key", right_on="col2", how="left")

display_side_by_side(df3, df4, df5, df6, df7, df8)

,col1,col2,col3,col4,key,vals
0,0,1,2,3,0,4
1,4,5,6,7,1,6
2,8,9,10,11,2,8
,col1,col2,col3,col4,key,vals
0,0,1,2,3,1.0,6.0
1,4,5,6,7,NaN,NaN
2,8,9,10,11,NaN,NaN
,col1,col2,col3,col4,key,vals
0,NaN,NaN,NaN,NaN,0,4
1,0.0,1.0,2.0,3.0,1,6


In [59]:
# Join on MultiIndex
df = pd.DataFrame(
    data=np.arange(12).reshape((4,3)),
    index=[["a", "a", "b", "b"], [1, 2] * 2],
    columns=[["col1", "col1", "col2"], ["ff", "gg", "ff"]]
)
df.index.names = [0, 1]
df2 = pd.DataFrame({"key":[0,1,2], "vals":[4,6,8]})

df3 = pd.merge(df, df2, left_on=df.index.names[1], right_on=df2.index, how="left").set_index(df.index)

display_side_by_side(df, df2, df3)

C:\Users\vovi\anaconda3\envs\dev2021\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


col1 
 col2 
 
 
 
 
 ff 
 gg 
 ff 
 
 
 0 
 1 
 
 
 
 
 
 
 
 a 
 1 
 0 
 1 
 2 
 
 
 2 
 3 
 4 
 5 
 
 
 b 
 1 
 6 
 7 
 8 
 
 
 2 
 9 
 10 
 11 
 
 
 
 
 
 
 key 
 vals 
 
 
 
 
 0 
 0 
 4 
 
 
 1 
 1 
 6 
 
 
 2 
 2 
 8 
 
 
 
 
 
 
 
 1 
 (col1, ff) 
 (col1, gg) 
 (col2, ff) 
 key 
 vals 
 
 
 0 
 1 
 
 
 
 
 
 
 
 
 
 
 a 
 1 
 1 
 0 
 1 
 2 
 1 
 6 
 
 
 2 
 2 
 3 
 4 
 5 
 2 
 8 
 
 
 b 
 1 
 1 
 6 
 7 
 8 
 1 
 6 
 
 
 2 
 2 
 9 
 10 
 11 
 2 
 8

In [60]:
# Get rows which are both in another DataFrame
df1 = pd.DataFrame(data={"col1":[1, 2, 3, 4, 5], "col2":[10, 11, 12, 13, 14]}) 
df2 = pd.DataFrame(data={"col1":[1, 2, 3], "col2":[10, 11, 12]})
df = df1[~df1.isin(df2)].dropna()
display_side_by_side(df1, df2, df)

,col1,col2
0,1,10
1,2,11
2,3,12
3,4,13
4,5,14
,col1,col2
0,1,10
1,2,11
2,3,12
,col1,col2


## Aggregating infromation in DataFrame

### Simple means, medians etc.

In [61]:
# Simple example
df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uuu"], index=["first", "second", "third", "fourth"])
df.iloc[:2,1] = np.nan
display(df)

# Mean by column, change axis=0 to get rows
display(df.mean(axis=1, skipna=False))
display(df.mean(axis=1, skipna=True))

,ff,ee,tt,uuu
first,0.021436,NaN,0.938259,0.402183
second,0.609567,NaN,0.121037,0.240753
third,0.137303,0.746673,0.574733,0.695427
fourth,0.559622,0.611344,0.961685,0.492085


first          NaN
second         NaN
third     0.538534
fourth    0.656184
dtype: float64

first     0.453959
second    0.323786
third     0.538534
fourth    0.656184
dtype: float64

### Apply means, medians etc. to subportion of data frame

In [62]:
# Easy way to calculate aggregations using lambdas and apply
df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uuu"], index=["first", "second", "third", "fourth"])
cols_to_apply = ["ff", "tt", "uuu"]
df2 = pd.DataFrame(df.apply(lambda x: np.max(x[cols_to_apply]), axis=1), columns=["Max of column subset"])
df3 = pd.DataFrame(df.apply(lambda x: np.mean(x[cols_to_apply]), axis=1), columns=["Mean of column subset"])

display_side_by_side(df, df2, df3)

ff 
 ee 
 tt 
 uuu 
 
 
 
 
 first 
 0.752928 
 0.351685 
 0.461548 
 0.428039 
 
 
 second 
 0.197018 
 0.014798 
 0.787131 
 0.085299 
 
 
 third 
 0.611171 
 0.024815 
 0.186124 
 0.606037 
 
 
 fourth 
 0.367947 
 0.878898 
 0.996342 
 0.074523 
 
 
 
 
 
 
 Max of column subset 
 
 
 
 
 first 
 0.752928 
 
 
 second 
 0.787131 
 
 
 third 
 0.611171 
 
 
 fourth 
 0.996342 
 
 
 
 
 
 
 Mean of column subset 
 
 
 
 
 first 
 0.547505 
 
 
 second 
 0.356483 
 
 
 third 
 0.467777 
 
 
 fourth 
 0.479604

### Group by in various forms

In [63]:
data = {"ID":[1, 1, 2], "Names":["Kate", "John", "Max"], "Age":[50, 25, 40], "Values":[3, 4, 8]}
df = pd.DataFrame(data=data, index=["first", "second", "first"])
df.index.name = "Index"

result1 = df.groupby(["ID"])["Values"].sum()
result1 = pd.DataFrame(result1)

result2 = df.groupby([df.index])["Age"].mean()
result2 = pd.DataFrame(result2)

result3 = df.groupby([df.index])["Age"].count()
result3 = pd.DataFrame(result3)

display_side_by_side(df,result1,result2,result3)

ID 
 Names 
 Age 
 Values 
 
 
 Index 
 
 
 
 
 
 
 
 
 first 
 1 
 Kate 
 50 
 3 
 
 
 second 
 1 
 John 
 25 
 4 
 
 
 first 
 2 
 Max 
 40 
 8 
 
 
 
 
 
 
 Values 
 
 
 ID 
 
 
 
 
 
 1 
 7 
 
 
 2 
 8 
 
 
 
 
 
 
 Age 
 
 
 Index 
 
 
 
 
 
 first 
 45 
 
 
 second 
 25 
 
 
 
 
 
 
 Age 
 
 
 Index 
 
 
 
 
 
 first 
 2 
 
 
 second 
 1

#### Table of counts/means for categorical variable

In [64]:
data = {"ID":[1, 1, 1, 2, 2], "Type":["moo", "boo", "boo", np.nan, "moo"], "Values":[3, 4, 6, 7, 4]}
df = pd.DataFrame(data=data)

# They magic touch is reset_index: count calculation is done on this new column!
df2 = df[["ID", "Type"]].reset_index().groupby(["ID", "Type"], as_index=False).count()
df2.columns = ["ID", "Type", "count"]

# Carefull with missing values! This works with them and assign them as -1
df3 = df[["ID", "Type"]].fillna(-1).reset_index().groupby(["ID", "Type"], as_index=False).count()
df3.columns = ["ID", "Type", "count"]

df4 = df[["ID","Type","Values"]].copy().groupby(["ID", "Type"], as_index=False).mean()
df4.columns = ["ID", "Type", "mean"]

display_side_by_side(df, df2, df3, df4)

,ID,Type,Values
0,1,moo,3
1,1,boo,4
2,1,boo,6
3,2,NaN,7
4,2,moo,4
,ID,Type,count
0,1,boo,2
1,1,moo,1
2,2,moo,1
,ID,Type,count


#### Table of counts for continuous variable

In [65]:
def continuous_binning(pdseries, bins=""):
    """
    Bins continious variable. Treats also NaNs
    Returns data frame.
    Arguments: 
        - pdseries: pandas.Series to be binned
        - bins (optional): user-defined bins. If
          not supplied then 5+1 bins by default.
    """
    df = pd.DataFrame()
    if bins == "":
        df["bins"] = pd.cut(pdseries, 5, right=True)        
    else:
        df["bins"] = pd.cut(pdseries, bins, right=True)
    df = df[["bins"]].reset_index().groupby(["bins"], as_index=False).count()
    df.columns = ["bins","count"]
    df_nans = pd.DataFrame({"bins":["NaN"], "count":[len(pdseries[np.isnan(pdseries)])]})
    df = df.append(df_nans, ignore_index=True)
    if df["count"].sum() != len(pdseries):
        raise ValueError("Something wrong in function continuous_binning!")
    return df, bins

df = pd.DataFrame(data = {"ID":[1, 2, 3,4, 5], "Values":[3.5, 4, 8.7, 12.6, 7]})
df2 = pd.DataFrame(data = {"ID":[1, 2, 3,4, 5], "Values":[3.5, 4, 8.7, 12.6, np.nan]})
bins = [0, 3, 6, 9, 12, 15, np.inf]

y = df["Values"]
df3,_ = continuous_binning(y, bins)

y = df2["Values"]
df4,_ = continuous_binning(y, bins)

display_side_by_side(df,df3,df4)

,ID,Values
0,1,3.5
1,2,4.0
2,3,8.7
3,4,12.6
4,5,7.0
,bins,count
0,"(0.0, 3.0]",0
1,"(3.0, 6.0]",2
2,"(6.0, 9.0]",2
3,"(9.0, 12.0]",0


In [66]:
# Select sum of first greatest N values within group
data = {"ID":[1, 2, 1, 2, 1], "Values":[3, 9, 8, 7, 3]}
df = pd.DataFrame(data=data)

df2 = df.copy()
df2.sort_values(["ID", "Values"], ascending=[True, False], inplace=True)
df2 = df2.groupby("ID").head(2)
df2 = pd.DataFrame(df2.groupby(["ID"])["Values"].sum().sort_values(ascending=False))

display_side_by_side(df, df2)

ID 
 Values 
 
 
 
 
 0 
 1 
 3 
 
 
 1 
 2 
 9 
 
 
 2 
 1 
 8 
 
 
 3 
 2 
 7 
 
 
 4 
 1 
 3 
 
 
 
 
 
 
 Values 
 
 
 ID 
 
 
 
 
 
 2 
 16 
 
 
 1 
 11

In [67]:
# Combination of group by and apply: caluclation between grouped columns
# Example: Total price of each product bought
df = pd.DataFrame({"Product":[1, 2, 1, 2], "Quantity":[1, 3, 2, 2], "Price":[10, 20, 5, 2]})
df2 = pd.DataFrame(df.groupby(["Product"]).apply(lambda df, a, b: sum(df[a] * df[b]), "Quantity", "Price"))
df2.columns = ["Total sell price"]
display_side_by_side(df, df2)

Product 
 Quantity 
 Price 
 
 
 
 
 0 
 1 
 1 
 10 
 
 
 1 
 2 
 3 
 20 
 
 
 2 
 1 
 2 
 5 
 
 
 3 
 2 
 2 
 2 
 
 
 
 
 
 
 Total sell price 
 
 
 Product 
 
 
 
 
 
 1 
 20 
 
 
 2 
 64

#### Weighted average with group by

In [68]:
df = pd.DataFrame({
    "name":["Foo", "Baar", "Foo", "Baar"],
    "score_1":[5, 10, 15, 10],
    "score_2" :[10, 15, 10, 25],
    "score_3" : [10, 20, 30, 40]
})
df = pd.DataFrame(df.groupby(["name", "score_1"]).apply(lambda x: np.average(x["score_2"], weights=x["score_3"])))
display(df)

0
name score_1           
Baar 10       21.666667
Foo  5        10.000000
     15       10.000000

#### SQL-like group by - sum combo

In [69]:
df = pd.DataFrame({"Product":[1, 2, 1, 2], "Quantity":[1, 3, 2, 2], "Price":[10, 20, 5, 2]})
df["counts_area"] = df["Price"].groupby(df["Product"]).transform("sum")
display(df)

,Product,Quantity,Price,counts_area
0,1,1,10,15
1,2,3,20,22
2,1,2,5,15
3,2,2,2,22


#### Group over datetime with different frequency

In [70]:
df = pd.DataFrame(
    data={"values":np.random.randn(8).cumsum()},
    index=pd.date_range("2009-10-01",
    periods=8,
    freq="1M"
))

display_side_by_side(
    df,
    df.groupby([
        pd.Grouper(freq="Q")
    ]).mean()
)

,values
2009-10-31,0.865321
2009-11-30,1.944501
2009-12-31,1.092606
2010-01-31,1.031157
2010-02-28,1.542034
2010-03-31,1.532391
2010-04-30,2.492854
2010-05-31,3.273650
,values
2009-12-31,1.300809


### Agg method

[Deprecate groupby.agg() with a dictionary when renaming](https://pandas.pydata.org/pandas-docs/stable/whatsnew/v0.20.0.html#whatsnew-0200-api-breaking-deprecate-group-agg-dict).

In [71]:
# Helper function
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = "percentile_%s" % n
    return percentile_

# Agg is similar to apply, but agg gives the flexibility of
# applying multiple functions at once. Agg() is handy at handling
# DataFrameGroupBy objects compared to apply()
df = pd.DataFrame({
    "name":["Foo", "Baar", "Foo", "Baar"],
    "score_1":[1, 2, 1, 4],
    "score_2" :[5, 10, 1, 10],
    "score_3" : [5, 10, 10, 10]
})

# Mean and sum of "score_2" grouped by "name" and "score_1" columns using apply
print(df.groupby(["name", "score_1"])["score_2"].apply(lambda x: x.mean()))
print(df.groupby(["name", "score_1"])["score_2"].apply(lambda x: x.sum()))

# Same with agg(), and even more aggregations applied to multiple columns, all at the same go!
df2 = df.groupby(["name", "score_1"]).agg({"score_2":[np.mean, np.sum, np.max, np.min], "score_3":[np.mean]}).copy()

# THIS IS PRIME EXAMPLE ON HOW TO CALCULATE SIMULTANEOUSLY CONDITIONAL DESCRIPTIVE STATISTICS!
# You can also pass in a lambd/customa functions to agg
# https://stackoverflow.com/questions/31521027/groupby-weighted-average-and-sum-in-pandas-dataframe
# Note that since depreceation of groupby.agg() with a dictionary when renaming, we have to manually
# columns resulting from a lambda function.
f_wmean = lambda x: np.average(x, weights=df.loc[x.index, "score_2"])
f_mean = lambda x: np.average(x)
f = {
    "score_2":["sum", "count", f_mean, percentile(50)],
    "score_3":{f_wmean, f_mean, "count"}
#    "score_3":{"mean_w":f_wmean, "mean":f_mean, "count":"count"} # this is the old syntax
}
df3 = df.groupby(["name", "score_1"]).agg(f).copy()

# One has to be careful if lambda functions are not same way around for different 1st level columns.
# Here first rename 2nd level column belonging to score_2, and only then use mass renamer for second
# column level
tuples = list(zip(
    df3.columns.get_level_values(0),
    [x if (y != "score_2" or x != "<lambda_0>") else "f_mean" for y, x in zip(df3.columns.get_level_values(0), df3.columns.get_level_values(1))]
))
df3.columns = pd.MultiIndex.from_tuples(tuples)
df3 = df3.rename(columns={"<lambda_0>":"f_wmean", "<lambda_1>":"f_mean"}, level=1)

display_side_by_side(df, df3)

name  score_1
Baar  2          10.0
      4          10.0
Foo   1           3.0
Name: score_2, dtype: float64
name  score_1
Baar  2          10
      4          10
Foo   1           6
Name: score_2, dtype: int64


name 
 score_1 
 score_2 
 score_3 
 
 
 
 
 0 
 Foo 
 1 
 5 
 5 
 
 
 1 
 Baar 
 2 
 10 
 10 
 
 
 2 
 Foo 
 1 
 1 
 10 
 
 
 3 
 Baar 
 4 
 10 
 10 
 
 
 
 
 
 
 
 score_2 
 score_3 
 
 
 
 
 sum 
 count 
 f_mean 
 percentile_50 
 count 
 f_wmean 
 f_mean 
 
 
 name 
 score_1 
 
 
 
 
 
 
 
 
 
 
 
 Baar 
 2 
 10 
 1 
 10 
 10 
 1 
 10.000000 
 10.0 
 
 
 4 
 10 
 1 
 10 
 10 
 1 
 10.000000 
 10.0 
 
 
 Foo 
 1 
 6 
 2 
 3 
 3 
 2 
 5.833333 
 7.5

### Pivot table

In [72]:
df = pd.DataFrame({
    "name":["Foo", "Baar", "Foo", "Baar"],
    "name2":["mm", "mm", "mm", "ee"],
    "score_1" :[10, 15, 10, 25],
    "score_2" : [10, 20, 30, 40]
})

# Basics
df2 = df.pivot_table(values="score_1", index="name", columns="name2", aggfunc=[np.mean,np.max]).copy()
df3 = df.pivot_table(values="score_1", index="name", columns="name2", aggfunc=[np.mean,np.min], margins=True).copy()

# Shares out of row sum
df4 = df.pivot_table(values="score_1", index="name", columns="name2", aggfunc=[np.sum], margins=True).copy()
df4.columns = df4.columns.get_level_values(1)
df4.drop(["All"], axis=0 , inplace=True)
df4.fillna(0, inplace=True)
df5 = df4.copy()
for _, row in df5.iterrows():
    row[0:-1] = row[0:-1] / row[-1]

display_side_by_side(df, df2, df3)
display_side_by_side(df4, df5)

name 
 name2 
 score_1 
 score_2 
 
 
 
 
 0 
 Foo 
 mm 
 10 
 10 
 
 
 1 
 Baar 
 mm 
 15 
 20 
 
 
 2 
 Foo 
 mm 
 10 
 30 
 
 
 3 
 Baar 
 ee 
 25 
 40 
 
 
 
 
 
 
 mean 
 amax 
 
 
 name2 
 ee 
 mm 
 ee 
 mm 
 
 
 name 
 
 
 
 
 
 
 
 
 Baar 
 25.0 
 15.0 
 25.0 
 15.0 
 
 
 Foo 
 NaN 
 10.0 
 NaN 
 10.0 
 
 
 
 
 
 
 mean 
 amin 
 
 
 name2 
 ee 
 mm 
 All 
 ee 
 mm 
 All 
 
 
 name 
 
 
 
 
 
 
 
 
 
 
 Baar 
 25.0 
 15.000000 
 20 
 25.0 
 15.0 
 15 
 
 
 Foo 
 NaN 
 10.000000 
 10 
 NaN 
 10.0 
 10 
 
 
 All 
 25.0 
 11.666667 
 15 
 25.0 
 10.0 
 10

name2,ee,mm,All
name,,,
Baar,25.0,15.0,40
Foo,0.0,20.0,20
name2,ee,mm,All
name,,,
Baar,25.0,15.0,40
Foo,0.0,20.0,20


### List observations with count > n

In [73]:
df = pd.DataFrame({
    "name":["Foo", "Baar", "Foo", "Foo"],
    "name2":["mm", "mm", "mm", "ee"],
    "score_1" :[10,15,10,25],
    "score_2" : [10,20,30,40]
})

counts = df["name"].value_counts()
counts  = counts[counts > 2].index

df[df["name"].isin(counts)]

,name,name2,score_1,score_2
0,Foo,mm,10,10
2,Foo,mm,10,30
3,Foo,ee,25,40


<h1>Data cleaning</h1>

## Find and replace string in data frame column

In [74]:
data = {"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Age":[50, 25, 41], "Values":[3, 4, 8]}
df = pd.DataFrame(data=data, columns=["ID", "Names", "Age", "Values", "Country"])

# find
df2 = df[df["Names"].str.contains("John")].copy()

# replace
df3 = df.copy()
df3["Names"] = df3["Names"].str.replace("John","Mike")

display_side_by_side(df, df2, df3)

,ID,Names,Age,Values,Country
0,1,Kate,50,3,NaN
1,2,John,25,4,NaN
2,3,Max,41,8,NaN
,ID,Names,Age,Values,Country
1,2,John,25,4,NaN
,ID,Names,Age,Values,Country
0,1,Kate,50,3,NaN
1,2,Mike,25,4,NaN
2,3,Max,41,8,NaN


## Replace garbage values with NaN

### Replace non-numerical values in numerical colun with NaN

In [75]:
df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uuu"], index=["first", "second", "third", "fourth"])
df.iloc[2,1] = "dd"
df.iloc[1,2] = "-----"
df.iloc[2,2] = "#SomeErrorVal"
df2 = df.copy()

# Count amount of non-numeric in a column
print("Column " + "ee" + " has {} non-numeric".format(len(df.loc[pd.to_numeric(df["ee"], errors="coerce").isnull(), "ee"])))

cols = ["ff", "ee", "tt", "uuu"]
for col in cols:
    df2[col][df2.apply(lambda x: np.isreal(x[col]) == True, axis=1) == False] = np.nan

display_side_by_side(df, df2)

Column ee has 1 non-numeric


<ipython-input-75-f9c15150e0ea>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col][df2.apply(lambda x: np.isreal(x[col]) == True, axis=1) == False] = np.nan
<ipython-input-75-f9c15150e0ea>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col][df2.apply(lambda x: np.isreal(x[col]) == True, axis=1) == False] = np.nan
<ipython-input-75-f9c15150e0ea>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col][df2.apply(lambda x

,ff,ee,tt,uuu
first,0.517909,0.918854,0.511667,0.555553
second,0.924492,0.36584,-----,0.832622
third,0.466254,dd,#SomeErrorVal,0.630345
fourth,0.576496,0.544354,0.23269,0.994473
,ff,ee,tt,uuu
first,0.517909,0.918854,0.511667,0.555553
second,0.924492,0.36584,NaN,0.832622
third,0.466254,NaN,NaN,0.630345
fourth,0.576496,0.544354,0.23269,0.994473


### Replace non-numerical values in string colun with NaN
Needs regex

In [76]:
data = {"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Values":["3", "4", "."]}
df = pd.DataFrame(data=data, columns=["ID", "Names", "Values"])

df2 = df.copy()
df2["Values"] = df2["Values"].replace(r"\.", np.nan, regex=True)

display_side_by_side(df, df2)

,ID,Names,Values
0,1,Kate,3
1,2,John,4
2,3,Max,.
,ID,Names,Values
0,1,Kate,3
1,2,John,4
2,3,Max,NaN


## Find NaN values from DataFrame

In [77]:
df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uuu"], index=["first", "second", "third", "fourth"])
df.iloc[2,1] = np.nan
df.iloc[0,0] = np.nan

# Calculate number of NaNs in column
print("Column " + "ee" + " has {} NaNs".format(pd.isnull(df["ee"]).sum()))

df2 = df[df.isnull().any(axis=1)].copy()

display_side_by_side(df, df2)

Column ee has 1 NaNs


,ff,ee,tt,uuu
first,NaN,0.847481,0.273539,0.668677
second,0.666127,0.997967,0.247489,0.379440
third,0.468435,NaN,0.574605,0.058540
fourth,0.420479,0.559183,0.224706,0.147116
,ff,ee,tt,uuu
first,NaN,0.847481,0.273539,0.668677
third,0.468435,NaN,0.574605,0.058540


## Clean NaN values

In [78]:
df = pd.DataFrame(np.random.rand(4,4), columns=["ff", "ee", "tt", "uuu"], index=["first", "second", "third", "fourth"])
df.iloc[2,1] = np.nan
df.iloc[1,3] = np.nan
df.iloc[0,0] = np.nan
df2 = df.copy()
df3 = df.copy()

# Fill NaN by mean of the column
df2["ee"].fillna(df["ee"].mean(), inplace=True)

# Fill using value from previuous row
df2["uuu"].fillna(method="bfill", inplace=True)

# Fill using value from next row
df2["ff"].fillna(method="bfill", inplace=True)

# Fill multiple columns by some scalar value
df3[["ee", "uuu", "ff"]] = df3[["ee", "uuu", "ff"]].fillna(0)

display_side_by_side(df, df2,df3)

,ff,ee,tt,uuu
first,NaN,0.793440,0.833470,0.755776
second,0.020317,0.430169,0.459506,NaN
third,0.089212,NaN,0.650462,0.671458
fourth,0.978180,0.819140,0.960231,0.754565
,ff,ee,tt,uuu
first,0.020317,0.793440,0.833470,0.755776
second,0.020317,0.430169,0.459506,0.671458
third,0.089212,0.680917,0.650462,0.671458
fourth,0.978180,0.819140,0.960231,0.754565
,ff,ee,tt,uuu


## Drop rows/columns with NaN values

In [79]:
df = pd.DataFrame([
    [np.nan, 2, np.nan, 0], [3, 4, np.nan, 1],
    [np.nan, np.nan, np.nan, 5],
    [7, 1, 6, 5]],
    columns=list("ABCD")
)

# Drop rows (axis=0)/ columns (axis=1) where all elements are NaN
df2 = df.dropna(axis=1, how="all").copy()

# Drop rows (axis=0)/ columns (axis=1) where any element is NaN
df3 = df.dropna(axis=0, how="any").copy()

# Keep rows (axis=0)/ columns (axis=1) which have at least thresh non-NaN values
df4 = df.dropna(axis=1, thresh=2).copy()

# Drop NAs in given columns with printgin what happens
df5 = df.copy()
for colname in df5.columns:
    print("Dropping {} observations due to NaN in {}".format(pd.isnull(df5[colname]).sum(), colname))
    df5 = df5[pd.isnull(df5[colname]) == False]

display_side_by_side(df, df2, df3, df4, df5)

Dropping 2 observations due to NaN in A
Dropping 0 observations due to NaN in B
Dropping 1 observations due to NaN in C
Dropping 0 observations due to NaN in D


,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,7.0,1.0,6.0,5
,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5
3,7.0,1.0,6.0,5
,A,B,C,D


## Drop duplicates

To only consider certain columns for identifying duplicates, use *subset* (by default use all of the columns).

*subset : column label or sequence of labels, optional*

 - keep : {‘first’, ‘last’, False}, default ‘first’
 - first : Mark duplicates as True except for the first occurrence.
 - last : Mark duplicates as True except for the last occurrence.
 - False : Mark all duplicates as True.

In [80]:
data = {"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Age":[50, 25, 45], "Values":[3, 8, 8]}
df = pd.DataFrame(data=data, columns=["ID", "Names", "Age", "Values", "Country"])

# Checks all columns for duplicate rows
df.duplicated(subset=None, keep="first")

# Checks subset of columns for duplicates
df.duplicated(subset=["Values"], keep="first")

# Drop duplicates
df2 = df.copy()
df2.drop_duplicates(subset=["Values"], keep="first", inplace=True)

display_side_by_side(df, df2)

,ID,Names,Age,Values,Country
0,1,Kate,50,3,NaN
1,2,John,25,8,NaN
2,3,Max,45,8,NaN
,ID,Names,Age,Values,Country
0,1,Kate,50,3,NaN
1,2,John,25,8,NaN


## Find nonnumeric values from string column

In [81]:
data = {"ID":[1, 2, 3], "Names":["Kate", "John", "Max"], "Values":["3", "4g", np.nan]}
df = pd.DataFrame(data=data, columns=["ID", "Names", "Values"])

# including NaNs
df2 = df[pd.to_numeric(df["Values"], errors="coerce").isnull()].copy()

# Exclude Nans
df3 = pd.DataFrame(df.loc[(pd.to_numeric(df["Values"], errors="coerce").isnull()) & (~pd.isnull(df["Values"])),"Values"])

display_side_by_side(df2, df3)

ID 
 Names 
 Values 
 
 
 
 
 1 
 2 
 John 
 4g 
 
 
 2 
 3 
 Max 
 NaN 
 
 
 
 
 
 
 Values 
 
 
 
 
 1 
 4g

# Working with time and dates

## Time and date data types

Pandas use 4 types of time functionalities
<ol>
  <li>Timestamp: a point in time</li>
  <li>Period: period spanning over time range</li>
  <li>DatetimeIndex</li>
  <li>PeriodIndex</li>
</ol>

<b> It is recommended to use yyyy-mm-dd convention for dates!</b> This avoids the danger of misinterpreting months vs. days.

In [82]:
# Both of these create a TimeStamp, but only latter accepts frontward European convention!
display(pd.Timestamp("2008-12-10"))
display(pd.to_datetime("10-12-2008", dayfirst=True, format="%d-%m-%Y")) # only to_datime has dayfirst option!

Timestamp('2008-12-10 00:00:00')

Timestamp('2008-12-10 00:00:00')

In [83]:
# Examples of periods
display(pd.Period("2013"))
display(pd.Period("2017Q1"))
display(pd.Period("05/2017"))
display(pd.Period("2017-12-10"))

Period('2013', 'A-DEC')

Period('2017Q1', 'Q-DEC')

Period('2017-05', 'M')

Period('2017-12-10', 'D')

In [84]:
# TimeStamp and Period indices in DataFrames
df1 = pd.DataFrame(np.arange(3), [pd.Timestamp("2008-12-10"), pd.Timestamp("2008-12-11"), pd.Timestamp("2008-12-12")])
df2 = pd.DataFrame(np.arange(3), [pd.Period("2008-12-10"), pd.Period("2008-12-11"), pd.Period("2008-12-12")])
df3 = pd.DataFrame(np.arange(3), [pd.Period("2008Q2"), pd.Period("2008Q3"), pd.Period("2008Q4")])
df4 = pd.DataFrame(np.arange(4), pd.date_range("2016-10-01", periods=4, freq="1D"))
df5 = pd.DataFrame(np.arange(4), pd.date_range("2016-10-01", periods=4, freq="3W-SUN")) #SUN specifies we want only sundays

display_side_by_side(df1,df2,df3,df4,df5)

,0
2008-12-10,0
2008-12-11,1
2008-12-12,2
,0
2008-12-10,0
2008-12-11,1
2008-12-12,2
,0
2008Q2,0
2008Q3,1


In [85]:
# Convert column into a PeriodIndex
df = pd.DataFrame({"Quarter":["2008-01-01","2008-04-01","2008-07-01"], "Values":[5,3,8]})
df2 = pd.DataFrame()
df2["Values"] = df["Values"]
df2.index = pd.PeriodIndex(pd.to_datetime(df["Quarter"]), freq="1Q")
df2.index.name = None

df3 = pd.DataFrame({"Year":["2008", "2009", "2010"], "Values":[5,3,8]})
df4 = pd.DataFrame()
df4["Values"] = df3["Values"]
df4.index = pd.PeriodIndex(pd.to_datetime(df3["Year"]), freq="1A")
df4.index.name = None

display_side_by_side(df, df2, df3, df4)

Quarter 
 Values 
 
 
 
 
 0 
 2008-01-01 
 5 
 
 
 1 
 2008-04-01 
 3 
 
 
 2 
 2008-07-01 
 8 
 
 
 
 
 
 
 Values 
 
 
 
 
 2008Q1 
 5 
 
 
 2008Q2 
 3 
 
 
 2008Q3 
 8 
 
 
 
 
 
 
 Year 
 Values 
 
 
 
 
 0 
 2008 
 5 
 
 
 1 
 2009 
 3 
 
 
 2 
 2010 
 8 
 
 
 
 
 
 
 Values 
 
 
 
 
 2008 
 5 
 
 
 2009 
 3 
 
 
 2010 
 8

In [86]:
#TimeDelta is the difference between two TimeStamps
display(pd.Timestamp("2008-12-10") - pd.Timestamp("2009-04-10"))

Timedelta('-121 days +00:00:00')

### Working with Period and PeriodIndex
These are slightly different creatures and one needs to be careful!

In [87]:
df = pd.DataFrame({"date":["2008-01-01","2009-01-01","2010-01-01"], "Values":[5,3,8]})
df.index = pd.PeriodIndex(pd.to_datetime(df["date"]), freq="1A")
df["date_p"] = pd.PeriodIndex(pd.to_datetime(df["date"]), freq="1A")

print("Index is here PeriodIndex but column date_p is Period!")
display(df)

thr = "2009"

print("For PeriodIndex stuff is pretty easy")
display(df[df.index < thr])
print(df.index.strftime("%Y"))

print("\n")

print("For Period stuff is harder!")
#df[df["date_p"] < thr] # this causes an error!
display(df[df["date_p"] < pd.Period(thr,"1A")]) # thr needs to be converted to Period as well!
print([x.strftime("%Y") for x in df["date_p"].values])

Index is here PeriodIndex but column date_p is Period!


,date,Values,date_p
date,,,
2008,2008-01-01,5,2008
2009,2009-01-01,3,2009
2010,2010-01-01,8,2010


For PeriodIndex stuff is pretty easy


,date,Values,date_p
date,,,
2008,2008-01-01,5,2008


Index(['2008', '2009', '2010'], dtype='object', name='date')


For Period stuff is harder!


,date,Values,date_p
date,,,
2008,2008-01-01,5,2008


['2008', '2009', '2010']


## Date columns in data frame

### Convert string column to datetime
https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime

In [88]:
df1 = pd.DataFrame(data={"value":[1, 2, 3], "dates":["01-01-2007", "01-01-2008","01-01-2009"]})
df2 = pd.DataFrame(data={"value":[1, 2, 3], "dates":["2007Q1", "2007Q2","2007Q1"]})

df3 = df.copy()
df3["dates"] =  pd.to_datetime(df1["dates"], format="%d-%m-%Y") 

df4 = df.copy()
df4["dates"] = pd.PeriodIndex(df2["dates"], freq='Q').to_timestamp()

display_side_by_side(df3, df4)
print("{} vs {}".format(df3["dates"].dtype, df4["dates"].dtype))

,date,Values,date_p,dates
date,,,,
2008,2008-01-01,5,2008,NaT
2009,2009-01-01,3,2009,NaT
2010,2010-01-01,8,2010,NaT
,date,Values,date_p,dates
date,,,,
2008,2008-01-01,5,2008,2007-01-01
2009,2009-01-01,3,2009,2007-04-01
2010,2010-01-01,8,2010,2007-01-01


datetime64[ns] vs datetime64[ns]


### Convert datetime to formatted string

In [89]:
data = {"value":[1, 2, 3], "dates":pd.to_datetime(["01-01-2007", "01-01-2008","01-01-2009"])}
df = pd.DataFrame(data=data)
df["dates"] = df["dates"].dt.strftime("%Y-%m-%d")
df

,value,dates
0,1,2007-01-01
1,2,2008-01-01
2,3,2009-01-01


### Harmonize dates in data frame column
Sometimes date column might not be harmonized enough to be converted directly to datetime.

In [90]:
"""
Top 2 answers here: https://stackoverflow.com/questions/23581128/how-to-format-date-string-via-multiple-formats-in-python
See also http://strftime.org/
"""
import dateutil.parser
from datetime import datetime

data = {"value":[1, 2, 3, 5, 8], "dates":["03/25/93", "04-12-2012", "Feb. 17, 2005", "6/2006", "2010"]}
df1 = pd.DataFrame(data=data)
df2 = df1.copy()

# User-defined function for version 2
def try_parsing_date(text):
    for fmt in (
        "%Y-%m-%d", "%m/%d/%y", "%m/%d/%Y", "%m-%d-%y", "%m-%d-%Y",
        "%d %b %Y", "%d %B %Y", "%B %d, %Y", "%b %d, %Y", "%B. %d, %Y",
        "%b %d %Y", "%b. %d, %Y", "%B %d %Y", "%B %Y", "%B, %Y",
        "%b, %Y", "%b %Y", "%m/%Y", "%Y", "%Y%m"):
        try:
            return datetime.strptime(text, fmt)
        except ValueError:
            pass
    print(text)
    raise ValueError("no valid date format found")

# This is version1 relying on dateutil.parser
# It is pretty good in general case, but might fail on
# very custom dates. Further, fallback treatment for eg. 
# missing days or months is werid; it takes current day/month!
datetimes1 = list()
for index, row in df2.iterrows():
    datetimes1.append(dateutil.parser.parse(row["dates"]))
df1["dates_dt"] = datetimes1

# Second version with user-defined function relies on
# datetime.strptime. It is more detailed as different date
# formats can be specified separately. Further, fallback treatment
# of missgin days/months is more convenient
datetimes2 = list()
for index, row in df2.iterrows():
    datetimes2.append(try_parsing_date(row["dates"]))
df2["dates_dt"] = datetimes2    

display_side_by_side(df1,df2)

,value,dates,dates_dt
0,1,03/25/93,1993-03-25
1,2,04-12-2012,2012-04-12
2,3,"Feb. 17, 2005",2005-02-17
3,5,6/2006,2006-06-04
4,8,2010,2010-03-04
,value,dates,dates_dt
0,1,03/25/93,1993-03-25
1,2,04-12-2012,2012-04-12
2,3,"Feb. 17, 2005",2005-02-17
3,5,6/2006,2006-06-01


## Time series tricks

### Basic operations

In [91]:
df = pd.DataFrame(
    data={"Values":np.random.randn(8).cumsum()},
    index=pd.date_range("2009-10-01",
    periods=8, freq="1W-THU")
)

# Nth differences
df2 = df.diff(2).copy()

# Downsample to monthly by mean (or sum, min, max)
df3 = df.resample("M").mean().copy()

# Upsample, and upfill
df3b = df.resample("D").ffill().copy()

# Upsample, use interpolation
df3c = df.resample("D").interpolate("linear").copy()

# Slice DataFrame to only include observations between some range
df4 = df["2009-10-09":"2009-11-07"].copy()

# Change frequency
df5 = df.asfreq("2W-THU").copy()

display_side_by_side(df, df2, df3, df3b.head(10), df3c.head(10), df4, df5)

,Values
2009-10-01,1.043956
2009-10-08,0.339000
2009-10-15,1.465235
2009-10-22,1.491763
2009-10-29,1.076352
2009-11-05,1.432162
2009-11-12,2.689258
2009-11-19,3.396660
,Values
2009-10-01,NaN


In [92]:
df = pd.DataFrame(
    data={"Values":np.random.randn(8).cumsum()},
    index=pd.date_range("2009-10-01",
    periods=8, freq="1W-THU")
)

# Lag and lead
df2 = df.shift(periods=2)
df3 = df.shift(periods=-2)

# Offset
df4 = df.copy()
df4.index = df4.index + pd.offsets.MonthEnd()

display_side_by_side(df, df2, df3, df4)

,Values
2009-10-01,0.203362
2009-10-08,-1.733950
2009-10-15,-2.307445
2009-10-22,-2.246305
2009-10-29,-2.974281
2009-11-05,-2.327079
2009-11-12,-0.214594
2009-11-19,-0.212268
,Values
2009-10-01,NaN


<h3>Get mean value per day over multiple years</h3>

In [93]:
from datetime import datetime
from datetime import timedelta

startdate = datetime(2004, 1, 1)
enddate = datetime(2008, 1, 1)
days = (enddate + timedelta(days=1) - startdate).days
data = np.random.random(days)
dates = [startdate + timedelta(days=x) for x in range(0, days)]

ts = pd.DataFrame(data, dates, columns=["values"])
ts.index.names = ["Day"]

# Average at each day over multiple years
day_averages = ts.groupby([ts.index.month, ts.index.day]).mean()
day_averages.index.names = ["Month", "Day"]
day_averages.columns = ["avg_values"]

display_side_by_side(ts.head(10), pd.DataFrame(day_averages).head(10))

,values
Day,
2004-01-01,0.063723
2004-01-02,0.573139
2004-01-03,0.466948
2004-01-04,0.002643
2004-01-05,0.760189
2004-01-06,0.571679
2004-01-07,0.161506
2004-01-08,0.824228
2004-01-09,0.039549


# Data frame styling

## Global stylings

### Setting number display format

In [94]:
pd.set_option("display.float_format", lambda x: "%.3f" % x)

## Style frame when displaying

In [95]:
def ColoringIROverSixPercent(val):
    color = "red" if val > 0.06 else "black"
    return "color: %s" % color

In [96]:
data = {"ID":[1, 2, 3],
        "Names":["firm1", "firm2", "firm3"],
        "InterestRate":[0.065, 0.11, 0.01],
        "MarketShare":[36, 5, 12],        
        "BalanceSheetTotal":[102000000, 45000, 8035700000]}
df = pd.DataFrame(data=data)

# Display styled data frame
df.style \
.format({
  "BalanceSheetTotal":"{:,.0f}".format,
  "InterestRate":"{:.1%}".format, 
  "MarketShare":lambda x: "{:.1%}".format(x/100),  
}) \
.applymap(ColoringIROverSixPercent, subset=pd.IndexSlice[:,["InterestRate"]])

,ID,Names,InterestRate,MarketShare,BalanceSheetTotal
0,1,firm1,6.5%,36.0%,"102,000,000"
1,2,firm2,11.0%,5.0%,"45,000"
2,3,firm3,1.0%,12.0%,"8,035,700,000"


## Style multi-index frame

In [97]:
df = pd.DataFrame(
    data=np.random.uniform(size=[4, 4]),
    index=[["a","a","b","b",], [1,2] * 2],
    columns=[["col1","col1","col2","col2"], ["ff","gg","ff","gg"]]
)

formats = {
    "ff":"{:.2f}",
    "gg":"{:.0%}"
}
formats_dict = {
    midx:formats[level_val]
    for level_val in formats
    for midx in [col for col in df if col[1]==level_val]
}
df.style.format(formats_dict)

# Uncategorized

## Re-index columns to start e.g from 100

In [98]:
df = pd.DataFrame({"Values":np.random.randn(8).cumsum()}
                  ,index=pd.date_range("2009-10-01"
                  , periods=8, freq="1W-THU"))
df2 = df.copy()
df2["type"] = "t1"
for col in df2.columns.values[:-1]:
    gg = df2.groupby("type")[col].apply(lambda x: x.div(x.iloc[0]).subtract(1))
    df2[col] = (1 + gg) * 100
df2.drop(["type"], axis=1, inplace=True)

display_side_by_side(df, df2)

,Values
2009-10-01,-0.799
2009-10-08,-2.482
2009-10-15,-2.996
2009-10-22,-3.061
2009-10-29,-1.990
2009-11-05,-0.607
2009-11-12,-0.562
2009-11-19,-0.902
,Values
2009-10-01,100.000


## Usual data describing stuff

In [99]:
data = {"ID":[1, 2, 3],"Names":["Kate", "John", "Max"], "Occupation":["CEO", "Doctor", "Driver"]
        ,"Vals1":[50, 25, 41], "Vals2":[3.0, 4.0, 8.0]}
df = pd.DataFrame(data=data)

print("Total length of data frame: {}".format(len(df)))

for name in df.columns:
    print("Column {} has {} non-numeric".format(name,len(df.loc[pd.to_numeric(df[name], errors="coerce").isnull(), name])))
    print("Column {} has {} NaNs".format(name,pd.isnull(df[name]).sum()))
    print("-"*15)

Total length of data frame: 3
Column ID has 0 non-numeric
Column ID has 0 NaNs
---------------
Column Names has 3 non-numeric
Column Names has 0 NaNs
---------------
Column Occupation has 3 non-numeric
Column Occupation has 0 NaNs
---------------
Column Vals1 has 0 non-numeric
Column Vals1 has 0 NaNs
---------------
Column Vals2 has 0 non-numeric
Column Vals2 has 0 NaNs
---------------


## Check monotonicity of column

In [100]:
data = {"ID":[1, 2, 3, 4, 5], "Names":["Kare", "John", "Max", "Mary", "Santa"], "Age":[50, 25, 41,99, 23], "Values":[3, 4, 8, 4, 2]}
df = pd.DataFrame(data=data)

print(df["ID"].is_monotonic_increasing)
print(df["ID"].is_monotonic_decreasing)
print(df["Age"].is_monotonic_increasing)

True
False
False
